In [1]:
from bs4 import BeautifulSoup
import pandas as pd

with open('../data/raw/1year_5g_mobile_details.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parsing HTML File
soup = BeautifulSoup(html_content, 'html.parser')

data = []
for product in soup.find_all('div', class_='sm-product'):
    # Extract brand and other details
    name_element = product.find('a', class_='name clamp-2')
    name = name_element.h2.text.split(' ')[0].strip() if name_element else None

    price_element = product.find('span', class_='price')
    price = int(price_element.text.replace('₹', '').replace(',', '').strip()) if price_element else None

    specs = product.find('ul', class_='sm-feat specs')
    if specs:
        details = specs.find_all('li')
        try:
            screen_size = float(details[4].text.split(',')[0].split(' ')[0].strip())
            ram_rom = details[2].text.split(',')
            ram = int(ram_rom[0].split('G')[0].strip())
            romExtract = lambda x: int(float(x.strip().split()[0]) * (1024 if 'TB' in x.upper() else 1))
            rom = romExtract(ram_rom[1].split('inbuilt')[0].strip())
            front_camera = float(details[5].text.split('&')[1].split('M')[0].strip())
            back_camera = int(details[5].text.split('&')[0].split('+')[0].split('M')[0].strip())
            battery = int(details[3].text.split('m')[0].strip())
            spec_score = int(product.find('div', class_='tags').find('b').text.strip())
        except IndexError:
            continue
    else:
        continue

    data.append({
        'Brand': name,
        'Price (Rs.)': price,
        'Screen Size (in)': screen_size,
        'Front Camera (MP)': front_camera,
        'Back Camera (MP)': back_camera,
        'Battery (mAh)': battery,
        'RAM (GB)': ram,
        'ROM (GB)': rom,
        'Spec Score': spec_score
    })

# Creating dataframe
df = pd.DataFrame(data)

output_csv_path = '../data/processed/5g_smartphones_dataset.csv'
df.to_csv(output_csv_path, index=False)